In [11]:
from torch import load
import torch
import sys
import numpy as np
from schnetpack.data import AtomsData
import schnetpack as spk
import ase.db
import ase.io
from ase.io import read,write
import joblib
import lightgbm as lgb
import os
from openbabel import openbabel
from rdkit import Chem

In [12]:
def load_model(target,path):
    if target!='PCE':
        model1=load(path, map_location=torch.device('cpu'))
        model1.eval()
        return model1
    else:
        model= joblib.load(path)
        
        return model
    

In [13]:
def cal_nd(mol):
    atoms=mol.toatoms()
    write('mol.xyz',atoms)
    obConversion = openbabel.OBConversion()
    obConversion.SetInAndOutFormats("xyz", "mol")
    mol = openbabel.OBMol()
    obConversion.ReadFile(mol, "mol.xyz")   # Open Babel will uncompress automatically
    mol.AddHydrogens() 
    obConversion.WriteFile(mol, '1.mol')

    #calculate Nd         
    mol = Chem.MolFromMolFile('1.mol')        
    n = len(mol.GetAtoms())         
    Nd = 0        
    for i in range(0,n):
        atom = mol.GetAtomWithIdx(i)
        #判断原子是否为芳香性
        if atom.GetIsAromatic() == True:
            Nd += 1
        if atom.GetIsAromatic() == False:
            #判断原子价电子是否等于总饱和度
            if atom.GetTotalValence() != atom.GetTotalDegree():
                Nd += 1
            if atom.GetTotalValence() == atom.GetTotalDegree():
                #判断原子是否在环上
                if atom.IsInRing() == True:
                    Nd += 1
        
        
    return Nd

In [14]:
def cal_prop(moln,molo,tag):
    
    
    al=.0
    if molo.data.Acceptor=='PC61BM':
        al= -3.70
        adl= 0.077824564
    if molo.data.Acceptor=='PC71BM':
        al= -3.91
        adl= 0.033470005
    if tag=='edahl':
        prop=al-float(molo.data.HOMO)
    if tag=='edall':
        prop=float(molo.data.LUMO)-al
    if tag=='adlumo':
        prop=adl
    if tag=='nd':
        prop=cal_nd(moln)


    return prop

In [15]:
def pred_data( model,tag,data):
     
            
    if tag== 'PCE':
        return pred_pce(model,data)
        
    else :
         return pred_prop(model,tag,data)    
             

In [16]:
def pred_pce(model,data):
    db=ase.db.connect(data)
    pce=[]
    ids=[]
    for row in db.select():
        x=[]
        x.extend((row.homo,row.lumo,row.edahl,row.edall,row.et1,row.nd,row.adlumo,row.dhomo,row.dlumo))
        y = model.predict(np.array(x).reshape(1,-1)).tolist()
#         print(y)
        pce.extend(y)
        ids.append(row.id)
        
    return ids,pce

In [17]:
def pred_prop(model,tag,data):
    pred=AtomsData(data)
    pred_loader = spk.AtomsLoader(pred, batch_size=10) #40!!
    
    for count, batch in enumerate(pred_loader):
        datapred = model(batch)
        ids=batch['_idx'].numpy().tolist()
        datapred=datapred[tag].detach().numpy().tolist()
        yield datapred,ids

In [18]:
def write_results(predata,tag,db):
    
    for num in predata.keys():
        for prop in predata[num].keys():
            
            db.update(id=num+1, **{prop: predata[num][prop]}) 
    
    return 0

In [19]:
def main():
    target=['et1','dhomo','dlumo','homo','lumo'] # need to predict with schnet
    target2=['nd','edahl','edall','adlumo'] # no need to predict
    predata={}
    db=ase.db.connect('pred.db')
    
    odb=ase.db.connect('radcap10000cepb3lyp2.db')
    for mol in odb.select():      
            atom=mol.toatoms()
            db.write(atom)
    for tag in target:
        best_model=load_model(target=tag,path='./package/'+tag+'_model')
    
        for property,id in pred_data(best_model,tag,data='pred.db'):
            for sid,sprop in zip(id,property):    
                predata.update({sid[0]:{tag:sprop[0]}})
        write_results(predata,tag,db)    
#         print(predata)
    for tag in target2:
        for moln,molo in zip(db.select(),odb.select()):
            sprop=cal_prop(moln,molo,tag)
            sid=moln.id-1
            predata.update({sid:{tag:sprop}})
           
        write_results(predata,tag,db)
    pcemodel=load_model(target='PCE',path='./package/lgb_model')
    
    ids,pce=pred_data(model=pcemodel,tag='PCE',data='pred.db')

    for sid,spce in zip(ids,pce):
        
        db.update(id=sid,PCE=spce)

    
    return 0

In [20]:
if __name__ == '__main__':
    status = main()
    
    

RDKit ERROR: [11:18:19] Explicit valence for atom # 10 N, 4, is greater than permitted


AttributeError: 'NoneType' object has no attribute 'GetAtoms'